In [204]:
from gensim.models import KeyedVectors


In [157]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_input_file="/Users/macbook/Downloads/glove.840B.300d 2.txt", word2vec_output_file="gensim_glove_vectors.txt")

(2196018, 300)

In [161]:
from gensim.models.keyedvectors import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format("/Users/macbook/PycharmProjects/nlp-course/workshops/gensim_glove_vectors.txt", binary=False)

KeyboardInterrupt: 

In [158]:
glove_model.most_similar(positive=["pianist", "cello"], negative=["piano"])


NameError: name 'glove_model' is not defined

In [160]:
glove_model.most_similar(positive=["traffic light", "sea"], negative=["color"])


NameError: name 'glove_model' is not defined

### Building NN with word2vec pre-trained vectors

#### Creating model

In [205]:
reviews_train = []
for line in open("/Users/macbook/aclImdb/movie_data/full_train.txt", "r"):
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open("/Users/macbook/aclImdb/movie_data/full_test.txt", "r"):
    reviews_test.append(line.strip())


In [206]:
train_target = [1 if i < 12500 else 0 for i in range(25000)]
test_target = [1 if i < 12500 else 0 for i in range(25000)]

In [207]:
import torch
import torch.nn as nn

#weight = torch.FloatTensor(w2vmodel.vectors)

class SentimentClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.fc1 = nn.Linear(embed_dim, embed_dim)
        self.fc2 = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        init_range = 0.5
        self.embedding.weight.data.copy_(torch.from_numpy(w2vmodel.vectors))
        self.fc1.weight.data.uniform_(-init_range, init_range)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-init_range, init_range)
        self.fc2.bias.data.zero_()

    def forward(self, text):
        embedded = self.embedding(text)
        fc1_output = self.fc1(embedded)
        pooled_output, _ = fc1_output.max(dim=1)
        return self.fc2(pooled_output)

In [208]:
from itertools import chain

train_tokens = list(chain(*[[token for token in sample.lower().split()] for sample in reviews_train]))

In [209]:
from collections import Counter

train_vocabulary = Counter(train_tokens)

In [210]:
UNKNOWN_TOKEN = "unknown"

index_to_token = [UNKNOWN_TOKEN] + list(train_vocabulary.keys())
token_to_index = {token: index + 1 for index, token in enumerate(train_vocabulary.keys())}
token_to_index[UNKNOWN_TOKEN] = 0
#

In [211]:

w2vmodel = KeyedVectors.load_word2vec_format ( "/Users/macbook/Downloads/GoogleNews-vectors-negative300.bin" , binary = True , limit = len(index_to_token) )


In [212]:
EMBEDDING_DIM = 300
BATCH_SIZE = 64
MAX_INPUT_LENGTH = 100

In [213]:
print(index_to_token[251637])
print(token_to_index["shite'"])

shite'
251637


In [214]:
def generate_batch(input_data, max_length = MAX_INPUT_LENGTH):
    texts = torch.tensor([padding(sample["text"], max_length) for sample in input_data], dtype=torch.long)
    labels = torch.tensor([sample["label"] for sample in input_data], dtype=torch.long)
    return texts, labels

def padding(text_tokens, max_length, padding_token = 0):
    if len(text_tokens) >= max_length:
        return text_tokens[:max_length]
    return text_tokens + [padding_token]*(max_length - len(text_tokens))


In [215]:
prepared_data = []

for label, text in zip(train_target, reviews_train):
    text_tokens = [token_to_index[token.lower()] for token in text.split()]
    prepared_data.append({"label": label, "text": text_tokens})

In [216]:
from torch.utils.data import DataLoader

def train(input_data):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(input_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)
    for i, (text, label) in enumerate(data):
        optimizer.zero_grad()
        text, label = text.to(device), label.to(device)
        output = model(text)
        loss = criterion(output, label)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == label).sum().item()

    return train_loss / len(input_data), train_acc / len(input_data)

def test(input_data):
    loss = 0
    acc = 0
    data = DataLoader(input_data, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, label in data:
        text, label = text.to(device), label.to(device)
        with torch.no_grad():
            output = model(text)
            loss = criterion(output, label)
            loss += loss.item()
            acc += (output.argmax(1) == label).sum().item()

    return loss / len(input_data), acc / len(input_data)

In [217]:
import random
import numpy

def set_seed(seed: int, n_gpu: int):
    random.seed(seed)
    numpy.random.seed(seed)
    torch.manual_seed(seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed)

In [218]:
set_seed(42, 1)

In [219]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [220]:
model = SentimentClassificationModel(vocab_size=len(index_to_token), embed_dim=EMBEDDING_DIM, num_class=len(set(train_target))).to(device)


In [228]:
import time
from torch.utils.data.dataset import random_split

N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

train_len = int(len(prepared_data) * 0.95)
train_data, validation_data = \
    random_split(prepared_data, [train_len, len(prepared_data) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train(train_data)
    valid_loss, valid_acc = test(validation_data)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')


#

Epoch: 1  | time in 2 minutes, 52 seconds
	Loss: 0.0053(train)	|	Acc: 80.2%(train)
	Loss: 0.0004(valid)	|	Acc: 80.4%(valid)
Epoch: 2  | time in 2 minutes, 40 seconds
	Loss: 0.0051(train)	|	Acc: 80.1%(train)
	Loss: 0.0008(valid)	|	Acc: 76.0%(valid)
Epoch: 3  | time in 2 minutes, 55 seconds
	Loss: 0.0050(train)	|	Acc: 79.9%(train)
	Loss: 0.0006(valid)	|	Acc: 81.4%(valid)
Epoch: 4  | time in 2 minutes, 31 seconds
	Loss: 0.0048(train)	|	Acc: 79.6%(train)
	Loss: 0.0006(valid)	|	Acc: 67.8%(valid)
Epoch: 5  | time in 2 minutes, 42 seconds
	Loss: 0.0047(train)	|	Acc: 79.3%(train)
	Loss: 0.0006(valid)	|	Acc: 71.7%(valid)


In [ ]:
w2vmodel["pianist"]


In [177]:
w2vmodel.most_similar(positive=["pianist", "cello"], negative=["piano"])


[('cellist', 0.8336730599403381),
 ('violinist', 0.8203256726264954),
 ('soloist', 0.7550286650657654),
 ('violist', 0.7540876269340515),
 ('Violinist', 0.7306528091430664),
 ('clarinetist', 0.7097249031066895),
 ('pianist_composer', 0.702890157699585),
 ('oboist', 0.701830267906189),
 ('viola', 0.7015129923820496),
 ('violinists', 0.6962666511535645)]

In [ ]:
type(w2vmodel.index2word), len(w2vmodel.index2word)



In [178]:
w2vmodel.most_similar(positive=["lighthouses"], negative=["lights"])



[('shipwrecks', 0.42038723826408386),
 ('lighthouse', 0.39504384994506836),
 ('Maritime_Museum', 0.3708776831626892),
 ('shipwreck', 0.35814881324768066),
 ('Mystic_Seaport', 0.3485428988933563),
 ('schooner', 0.3406549394130707),
 ('schooners', 0.3327274024486542),
 ('Shipwreck', 0.3270357847213745),
 ('lighthouse_keeper', 0.3245318531990051),
 ('dockyard', 0.31617963314056396)]

In [182]:
w2vmodel.most_similar(positive=["clock", "temperature"], negative=["hours"])


[('thermometer', 0.5081078410148621),
 ('Temperature', 0.49295130372047424),
 ('temperatures', 0.4852280914783478),
 ('ambient_temperature', 0.47066593170166016),
 ('thermostat', 0.4558141231536865),
 ('clocks', 0.43162035942077637),
 ('Clock', 0.4239221215248108),
 ('thermometers', 0.4135264754295349),
 ('humidity', 0.407976895570755),
 ('temperature_humidity', 0.39861053228378296)]

In [186]:
w2vmodel.most_similar(positive=["cop", "chess"], negative=["gun"])


[('grandmasters', 0.47483304142951965),
 ('grandmaster', 0.46184688806533813),
 ('Chess', 0.4280867576599121),
 ('detective', 0.3878359794616699),
 ('jewel_thief', 0.3797242045402527),
 ('pickpocket', 0.37929707765579224),
 ('Chess_Federation', 0.3768908977508545),
 ('Scrabble', 0.37643611431121826),
 ('Sasikiran', 0.3756202757358551),
 ('sleuth', 0.3712921142578125)]

In [192]:
w2vmodel.most_similar(positive=["USA"], negative=["Trump"])


[('Germany', 0.32618555426597595),
 ('Great_Britain', 0.324604332447052),
 ('Unites_States', 0.3197040557861328),
 ('Netherlands', 0.31571221351623535),
 ('United_States', 0.3146864175796509),
 ('Belgium', 0.3079031705856323),
 ('Canada', 0.2991408109664917),
 ('Usa', 0.2970855236053467),
 ('United_Kingdom', 0.28943854570388794),
 ('Australia', 0.2836306691169739)]

In [229]:
prepared_test_data = []

for label, text in zip(test_target, reviews_test):
    text_tokens = [token_to_index.get(token.lower(), 0) for token in text.split()]
    prepared_test_data.append({"label": label, "text": text_tokens})

In [230]:
test(prepared_test_data)


(tensor(2.7093e-05), 0.59628)